In [4]:
import os, sys
sys.path.append('..')
from utility import *
import matplotlib.pyplot as plt
import numpy as np

# Hyper Parameters 
input_size    = 784
hidden_size   = 500
num_classes   = 10
num_epochs    = 5
batch_size    = 100
learning_rate = 0.001
num_model     = 0

# MNIST Dataset 
train_dataset, test_dataset = fetch_mnist_data(dir='../data')

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          batch_size=batch_size, 
                                          shuffle=False)

exec('net = CNN_Net%s().cuda().double() ' % num_model)
    

print("num of model params %d" % model_params(net))
    
criterion = nn.CrossEntropyLoss().cuda() 
optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)  

##################################################################################################
loss_total = []
accu_total = []
for epoch in range(num_epochs):
    running_loss = 0.0
    running_correct = 0
    
    test_loss = 0.0
    for i, (images, labels) in enumerate(train_loader):  
        # Convert torch tensor to Variable
        #images = Variable(images.view(-1, 28*28))
        images = Variable(images.type(torch.DoubleTensor)).cuda()
        labels = Variable(labels.type(torch.LongTensor)).cuda()
        
        optimizer.zero_grad()
        outputs = net(images)
        _, pred = torch.max(outputs.data, 1)
        
        loss = criterion(outputs, labels)
        loss.backward()
    
        running_correct += torch.sum(pred == labels.data)
        running_loss += loss.data[0]
        
        optimizer.step()
        
    cur_accu = running_correct/len(train_loader)
    cur_loss = running_loss/len(train_loader)
    
    loss_total.append(cur_loss)
    accu_total.append(cur_accu)
    print ('Epoch [%d/%d], Accu:%.4f Loss: %.4f' % (epoch+1, num_epochs, cur_accu, cur_loss))
    

plt.figure(1)
plt.plot(loss_total, 'k')
plt.figure(2)
plt.plot(accu_total, 'k')        
plt.show()
    
# Test the Model
correct = 0
total = 0
for images, labels in test_loader:
    #images = Variable(images.view(-1, 28*28))
    images = Variable(images.type(torch.DoubleTensor)).cuda()
    outputs = net(images)
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted.cpu().numpy() == labels).sum()

print('Accuracy of the network on the 10000 test images: %d %%' % (100 * correct / total))

# Save the Model
torch.save(net.state_dict(), 'model.pkl') #net.load_state_dict(torch.load('model.pkl'))

num of model params 21840
torch.Size([100, 1, 28, 28])
torch.Size([100, 1, 28, 28])
torch.Size([100, 1, 28, 28])
torch.Size([100, 1, 28, 28])
torch.Size([100, 1, 28, 28])
torch.Size([100, 1, 28, 28])
torch.Size([100, 1, 28, 28])
torch.Size([100, 1, 28, 28])
torch.Size([100, 1, 28, 28])
torch.Size([100, 1, 28, 28])
torch.Size([100, 1, 28, 28])
torch.Size([100, 1, 28, 28])
torch.Size([100, 1, 28, 28])
torch.Size([100, 1, 28, 28])
torch.Size([100, 1, 28, 28])
torch.Size([100, 1, 28, 28])
torch.Size([100, 1, 28, 28])
torch.Size([100, 1, 28, 28])
torch.Size([100, 1, 28, 28])
torch.Size([100, 1, 28, 28])
torch.Size([100, 1, 28, 28])
torch.Size([100, 1, 28, 28])
torch.Size([100, 1, 28, 28])
torch.Size([100, 1, 28, 28])
torch.Size([100, 1, 28, 28])
torch.Size([100, 1, 28, 28])
torch.Size([100, 1, 28, 28])
torch.Size([100, 1, 28, 28])
torch.Size([100, 1, 28, 28])
torch.Size([100, 1, 28, 28])
torch.Size([100, 1, 28, 28])
torch.Size([100, 1, 28, 28])
torch.Size([100, 1, 28, 28])
torch.Size([100, 

KeyboardInterrupt: 

In [1]:
from graphviz import Digraph
import torch
from torch.autograd import Variable


def make_dot(var, params):
    """ Produces Graphviz representation of PyTorch autograd graph
    
    Blue nodes are the Variables that require grad, orange are Tensors
    saved for backward in torch.autograd.Function
    
    Args:
        var: output Variable
        params: dict of (name, Variable) to add names to node that
            require grad (TODO: make optional)
    """
    param_map = {id(v): k for k, v in params.items()}
    print(param_map)
    
    node_attr = dict(style='filled',
                     shape='box',
                     align='left',
                     fontsize='12',
                     ranksep='0.1',
                     height='0.2')
    dot = Digraph(node_attr=node_attr, graph_attr=dict(size="12,12"))
    seen = set()
    
    def size_to_str(size):
        return '('+(', ').join(['%d'% v for v in size])+')'

    def add_nodes(var):
        if var not in seen:
            if torch.is_tensor(var):
                dot.node(str(id(var)), size_to_str(var.size()), fillcolor='orange')
            elif hasattr(var, 'variable'):
                u = var.variable
                node_name = '%s\n %s' % (param_map.get(id(u)), size_to_str(u.size()))
                dot.node(str(id(var)), node_name, fillcolor='lightblue')
            else:
                dot.node(str(id(var)), str(type(var).__name__))
            seen.add(var)
            if hasattr(var, 'next_functions'):
                for u in var.next_functions:
                    if u[0] is not None:
                        dot.edge(str(id(u[0])), str(id(var)))
                        add_nodes(u[0])
            if hasattr(var, 'saved_tensors'):
                for t in var.saved_tensors:
                    dot.edge(str(id(t)), str(id(var)))
                    add_nodes(t)
    add_nodes(var.grad_fn)
    return dot

from torchvision import models
inputs = torch.randn(1,3,224,224)
resnet18 = models.resnet18()
y = resnet18(Variable(inputs))
# print(y)

g = make_dot(y, resnet18.state_dict())
g.view()

{140412363526472: 'conv1.weight', 140412363526920: 'bn1.weight', 140412363526984: 'bn1.bias', 140412363527048: 'bn1.running_mean', 140412363527112: 'bn1.running_var', 140412363528712: 'layer1.0.conv1.weight', 140412363529160: 'layer1.0.bn1.weight', 140412363529224: 'layer1.0.bn1.bias', 140412363529288: 'layer1.0.bn1.running_mean', 140412363529352: 'layer1.0.bn1.running_var', 140412363530184: 'layer1.0.conv2.weight', 140412363473352: 'layer1.0.bn2.weight', 140412363473416: 'layer1.0.bn2.bias', 140412363473480: 'layer1.0.bn2.running_mean', 140412363473544: 'layer1.0.bn2.running_var', 140412363474376: 'layer1.1.conv1.weight', 140412363474824: 'layer1.1.bn1.weight', 140412363474888: 'layer1.1.bn1.bias', 140412363474952: 'layer1.1.bn1.running_mean', 140412363475016: 'layer1.1.bn1.running_var', 140412363475848: 'layer1.1.conv2.weight', 140412363476296: 'layer1.1.bn2.weight', 140412363476360: 'layer1.1.bn2.bias', 140412363476424: 'layer1.1.bn2.running_mean', 140412363476488: 'layer1.1.bn2.run

'Digraph.gv.pdf'

In [4]:
net.load_state_dict(torch.load('model.pkl'))
    
correct = 0
total = 0
for images, labels in test_loader:
    #images = Variable(images.view(-1, 28*28))
    images = Variable(images.type(torch.DoubleTensor)).cuda()
    outputs = net(images)
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted.cpu().numpy() == labels).sum()

print('Accuracy of the network on the 10000 test images: %d %%' % (100 * correct / total))

FileNotFoundError: [Errno 2] No such file or directory: 'model.pkl'